In [1]:
!git clone https://github.com/1vlex/hw3-tts-vocoder.git
%cd hw3-tts-vocoder

Cloning into 'hw3-tts-vocoder'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 66 (delta 16), reused 49 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 23.08 KiB | 11.54 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/hw3-tts-vocoder


In [2]:
!python -m pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.0 MB/s eta 0:00:00


In [3]:
CKPT_DRIVE_URL = "https://drive.google.com/file/d/197YvUjKbh2pW9jAg6XH4PDw4Lu1gRS3R/view?usp=sharing"

!python scripts/download_checkpoints.py --url "{CKPT_DRIVE_URL}" --out checkpoints/best_generator.pt
!ls -lh checkpoints

Downloading...
From (original): https://drive.google.com/uc?id=197YvUjKbh2pW9jAg6XH4PDw4Lu1gRS3R
From (redirected): https://drive.google.com/uc?id=197YvUjKbh2pW9jAg6XH4PDw4Lu1gRS3R&confirm=t&uuid=fa6602f5-8f7c-4904-ae81-4e30f234839b
To: /content/hw3-tts-vocoder/checkpoints/best_generator.pt
100% 55.8M/55.8M [00:01<00:00, 54.4MB/s]
[OK] Downloaded checkpoint to checkpoints/best_generator.pt (53.18 MB)
total 54M
-rw-r--r-- 1 root root 54M Feb 21 10:50 best_generator.pt


In [4]:
!mkdir -p data/mos_eval/audio data/mos_eval/transcriptions
!python -m pip install gdown -q
!gdown --folder "https://drive.google.com/drive/folders/1oClqdJjEGq6nDl9pAhqXivXGD5uJw1RJ?usp=sharing" -O data/mos_eval/audio
!find data/mos_eval -maxdepth 3 -type f | sort

Retrieving folder contents
Processing file 1dy_mNrvnJ8JLYW7sqRV0Jt9vvz-sz2iR 1.wav
Processing file 1whJ4pNziVZh6L9Z3WPp-sNizGBv5yKiC 2.wav
Processing file 1WxH6KZ8ORyWzAO-FXZaGOt0-X6JihJbe 3.wav
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dy_mNrvnJ8JLYW7sqRV0Jt9vvz-sz2iR
To: /content/hw3-tts-vocoder/data/mos_eval/audio/1.wav
100% 297k/297k [00:00<00:00, 123MB/s]
Downloading...
From: https://drive.google.com/uc?id=1whJ4pNziVZh6L9Z3WPp-sNizGBv5yKiC
To: /content/hw3-tts-vocoder/data/mos_eval/audio/2.wav
100% 477k/477k [00:00<00:00, 65.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WxH6KZ8ORyWzAO-FXZaGOt0-X6JihJbe
To: /content/hw3-tts-vocoder/data/mos_eval/audio/3.wav
100% 495k/495k [00:00<00:00, 123MB/s]
Download completed
data/mos_eval/audio/1.wav
data/mos_eval/audio/2.wav
data/mos_eval/audio/3.wav


In [5]:
texts = {
    "1.txt": "Ожидал увидеть тут толпы игроков, жаждущих меня убить, но никому моя голова почему-то не потребовалась",
    "2.txt": "Уверяю вас, сэр, большую часть своей жизни я был глубоко убежден, что любой мир состоит из чего угодно, только не из добрых людей",
    "3.txt": "Наш самый суровый священник был абсолютно никакой, в хлам, как распоследний питерский сантехник в канун дня праздника Парижской коммуны",
}

import os
os.makedirs("data/mos_eval/transcriptions", exist_ok=True)

for name, text in texts.items():
    with open(f"data/mos_eval/transcriptions/{name}", "w", encoding="utf-8") as f:
        f.write(text)

print("Transcriptions created:")
print(os.listdir("data/mos_eval/transcriptions"))

Transcriptions created:
['2.txt', '1.txt', '3.txt']


In [6]:
!python synthesize.py \
  infer.checkpoint_path=./checkpoints/best_generator.pt \
  infer.input_dir=./data/mos_eval \
  infer.output_dir=./outputs/mos_eval \
  infer.mode=resynthesis

audio:
  sample_rate: 22050
  win_length: 1024
  hop_length: 256
  n_fft: 1024
  f_min: 0
  f_max: 11025
  n_mels: 80
  power: 2.0
model:
  generator:
    upsample_rates:
    - 8
    - 8
    - 2
    - 2
    upsample_kernel_sizes:
    - 16
    - 16
    - 4
    - 4
    upsample_initial_channel: 512
infer:
  checkpoint_path: ./checkpoints/best_generator.pt
  input_dir: ./data/mos_eval
  output_dir: ./outputs/mos_eval
  mode: resynthesis

[OK] /content/hw3-tts-vocoder/outputs/mos_eval/1_gen.wav
[OK] /content/hw3-tts-vocoder/outputs/mos_eval/2_gen.wav
[OK] /content/hw3-tts-vocoder/outputs/mos_eval/3_gen.wav
[DONE] Inference completed


In [7]:
from IPython.display import Audio, display
import os

for p in ["outputs/mos_eval/1_gen.wav", "outputs/mos_eval/2_gen.wav", "outputs/mos_eval/3_gen.wav"]:
    print(p, "| exists:", os.path.exists(p))
    display(Audio(p))

outputs/mos_eval/1_gen.wav | exists: True


outputs/mos_eval/2_gen.wav | exists: True


outputs/mos_eval/3_gen.wav | exists: True
